## - Jean Durán Villanueva
##- 316032416
##- jean.dv@ciencias.unam.mx

## Funciones tomadas del notebook "1_phonetics"

In [66]:
import requests as r

response = r.get("https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/en_US.txt")

### Obteniendo el corpus

In [67]:
lang_codes = {
  "ar": "Arabic (Modern Standard)",
  "de": "German",
  "en_UK": "English (Received Pronunciation)",
  "en_US": "English (General American)",
  "eo": "Esperanto",
  "es_ES": "Spanish (Spain)",
  "es_MX": "Spanish (Mexico)",
  "fa": "Persian",
  "fi": "Finnish",
  "fr_FR": "French (France)",
  "fr_QC": "French (Québec)",
  "is": "Icelandic",
  "ja": "Japanese",
  "jam": "Jamaican Creole",
  "km": "Khmer",
  "ko": "Korean",
  "ma": "Malay (Malaysian and Indonesian)",
  "nb": "Norwegian Bokmål",
  "nl": "Dutch",
  "or": "Odia",
  "ro": "Romanian",
  "sv": "Swedish",
  "sw": "Swahili",
  "tts": "Isan",
  "vi_C": "Vietnamese (Central)",
  "vi_N": "Vietnamese (Northern)",
  "vi_S": "Vietnamese (Southern)",
  "yue": "Cantonese",
  "zh": "Mandarin"
}
iso_lang_codes = list(lang_codes.keys())

In [68]:
def response_to_dict(ipa_list: list) -> dict:
    """Parse to dict the list of word-IPA

    Each element of text hae the format:
    [WORD][TAB][IPA]

    Parameters
    ----------
    ipa_list: list
        List with each row of ipa-dict raw dataset file

    Returns
    -------
    dict:
        A dictionary with the word as key and the phonetic
        representation as value
    """
    result = {}
    for item in ipa_list:
        item_list = item.split("\t")
        result[item_list[0]] = item_list[1]
    return result

def get_ipa_dict(iso_lang: str) -> dict:
    """Get ipa-dict file from Github

    Parameters:
    -----------
    iso_lang:
        Language as iso code

    Results:
    --------
    dict:
        Dictionary with words as keys and phonetic representation
        as values for a given lang code
    """
    response = r.get(f"https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/{iso_lang}.txt")
    raw_data = response.text.split("\n")
    return response_to_dict(raw_data[:-1])

In [69]:
from collections import defaultdict
lang = "es_MX"

# Práctica

## Agregar un nuevo modo de búsqueda donde se extienda el comportamiento básico del buscador para ahora buscar por frases. Ejemplo:




```
[es_MX]>>> Hola que hace
 /ola/ /ke/ /ase/
```

Hecho ✅. Resultado en la funcion "f" al final de este notebook

## Optimizar el código para agregar los datasets en un "cache" a demanda y no descargar todo el corpus de un trancazo. Esto quiere decir que al inicio de la ejecución no habrá ningun dataset descargado. Mientras la usuaria vaya seleccionado idiomas los irá agregando a un cache local (puede ser persistente o en memoria). Ejemplo:

```
lang>> es_MX
Corpus no encontrado. Descargando ...
[es_MX]>>
...
lang>> en_US
Corpus no encontrado. Descargando ...
[en_US]>>
...
lang>> es_MX
[es_MX]>>
...
```

### Esta funcion se encarga de agregar un lenguaje a la vez.


In [70]:
import requests as r
response = r.get("https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/en_US.txt")

def get_lang(dict, iso_lang):
    """Agrega un nuevo idioma a nuestro dataset

    dict
      el diccionario al que agregamos un nuevo lenguaje

    iso_lang
      aque iso que agregaremos
    """
    leng = get_ipa_dict(iso_lang)

    if leng == {}:
      #print("El idioma que buscas no existe")
      return False
    else:
      dict[iso_lang] = leng
      return dict

## EXTRA

- Mejorar la solución al escenario cuando no se encuentran las palabras en el dataset mostrando palabras similares. Ejemplo:

```
[es_MX]>> pero
No se encontro <<pero>> en el dataset. Palabras aproximadas:
perro /pero/
perno /peɾno/
[es_MX]>>
```

##  💻💻💻💻💻💻Código 💻💻💻💻💻💻




### "get_ipa_transcriptions"

- Toma una palabra y un lenguaje. Regresa el ipa de esta
- Inspirada en la funcion "get_ipa_transcriptions" del notebook "1_phonetics"
- La diferencia radica en que esta funcion regresa una subcadena que sea parte del lenguaje
  - En caso de no encontrar a la cadena, claro.
  - Las subcadenas que busca son del estilo: si no encuentra computadorabcd, entonces computadorabc, computadorab, y regresa computadora
    - Esto lo hace hasta la mitad de la longitud de la cadena. Si osoabc, entonces busca hasta oso
- Está preparada además para regresar un mensaje en caso de no haber encontrado una subcadena

In [71]:
def get_ipa_transcriptions(word: str, sub_dict: dict) -> list[str]:
  word.lower()
  try:
    return (sub_dict[word]).split(", ") #1a. salida. Todo igual
  except:
    #aviso = "NOT FOUND. Here it is some words related. Lets try with a subchain."
    l = int(len(word)/2)
    for i in range(1,l):
      try:
        result = sub_dict[word[:-i]]
        word = word[:-i]
        return (True,result.split(", "),word) #2a salida. Cuando encontro sub cadena.
      except:
        pass
  aviso2 = "NOT FOUND. Let's try with near subchains...      Near subchains NOT FOUNDED"
  return (False,aviso2) #3a salida. Cuando no encontro subcadena

### "phrase_to_ipa"

- Se encarga de mostrar los resultados de cada frase que se escribe en un mismo idioma, hasta que se salga de esta funcion
- Pide la frase al usuario y llama mediante un for a "get_ipa_transcriptions" con el lenguaje y cada palabra.
  - Se guardan los resultados y se despliegan mediante un for (un poco ineficiente ahora que lo pienso, jeje).
    - Dependiendo si se encontro la cadena, se encontro una subcadena o no se despliegan los mensahes  

In [72]:
def phrase_to_ipa(sub_dict):
  query = input(f"\n  phrase>> ")
  query = query.split(" ")
  results = [get_ipa_transcriptions(q, sub_dict) for q in query]
  for q,r in zip(query, results):
    if not isinstance(r, tuple):
      print("  <<<<<<<<>>>>>>>>>")
      print("  ",q, " | ", ", ".join(r)) #word exists
    else:
      if r[0]:
        print("  <<<<<<<<word not found, but subchain of word>>>>>>>>>")
        print("  ",r[2], " | ", ", ".join(r[1])) #subchain of word exists
      else:
        print("  <<<<<<<<>>>>>>>>>")
        print("  ",q, " | ",r[1]) #subchain of word doesnt exists


  while query:
    sub_dict
    query = input(f"\n  phrase>> ")
    if query == "":
      print("\n")
      break
    query = query.split(" ")
    results = [get_ipa_transcriptions(q, sub_dict) for q in query]
    for q,r in zip(query, results):
      if not isinstance(r, tuple):
        print("  <<<<<<<<>>>>>>>>>")
        print("  ",q, " | ", ", ".join(r)) #word exists
      else:
        if r[0]:
          print("  <<<<<<<<word not found, but subchain of word>>>>>>>>>")
          print("  ",r[2], " | ", ", ".join(r[1])) #subchain of word exists
        else:
          print("  <<<<<<<<>>>>>>>>>")
          print("  ",q, " | ",r[1]) #subchain of word doesnt exists

### Funcion f, que cumple los 3 puntos de la practica

- Acepta frases, i.e., más de una cadena
- Agrega los lenguajes a demanda, ie,
  - No precarga ellos sino que los descarga si no lo tiene y lo mantiene incluso si se vuelve a ejecutar la funcion "f"
- Si no encuentra la cadena, sugiere subcadenas de esta.
  - Muestra la primer subcadena que encuentre y si no la encuentra, muestra un mensaje indicandolo

In [73]:
def f(dict):
  print("Representación fonética de palabras")

  print(f"Lenguas disponibles: {(iso_lang_codes)} \n \n \n")

  iso = " "
  while iso:
    iso = input("lang>> ")
    if iso == "":
      print("Bye bye 👋🏼")
      break

    try:
      sub_dict = dict[iso]
      #proceso de escribir la frase
      phrase_to_ipa(sub_dict)
    except:
      print("El idioma no se encuentra de manera local. Consultandola...")
      result = get_lang(dict, iso)
      if not result:
        print("No se encuentra el iso <",iso,"> que ingresas. Intenta con otro")
      else:
        print("Encontrado y guardado!")
        #proceso de escribir la frase
        sub_dict = dict[iso]
        phrase_to_ipa(sub_dict)

Declarar el diccionario "dict" por separado permite que se cuando se vuelva a ejecutar la funcion "f(dict)" en el futuro, los idiomas se queden guardados. (Todo esto mientras dict no se iguale a el diccionario vacío).

In [74]:
dict = {}

Para probar todo el codigo anterior, basta con ejecutar la siguiente linea de código

In [75]:
f(dict)

Representación fonética de palabras
Lenguas disponibles: ['ar', 'de', 'en_UK', 'en_US', 'eo', 'es_ES', 'es_MX', 'fa', 'fi', 'fr_FR', 'fr_QC', 'is', 'ja', 'jam', 'km', 'ko', 'ma', 'nb', 'nl', 'or', 'ro', 'sv', 'sw', 'tts', 'vi_C', 'vi_N', 'vi_S', 'yue', 'zh'] 
 
 

lang>> fr_
El idioma no se encuentra de manera local. Consultandola...
No se encuentra el iso < fr_ > que ingresas. Intenta con otro
lang>> fr_FR
El idioma no se encuentra de manera local. Consultandola...
Encontrado y guardado!

  phrase>> j'aime tes genoux
  <<<<<<<<word not found, but subchain of word>>>>>>>>>
   j'ai  |  /ʒe/
  <<<<<<<<>>>>>>>>>
   tes  |  /te/
  <<<<<<<<>>>>>>>>>
   genoux  |  /ʒənu/

  phrase>> paris
  <<<<<<<<>>>>>>>>>
   paris  |  /paʁi/

  phrase>> 


lang>> es_MX
El idioma no se encuentra de manera local. Consultandola...
Encontrado y guardado!

  phrase>> viva la vida
  <<<<<<<<>>>>>>>>>
   viva  |  /biβa/
  <<<<<<<<>>>>>>>>>
   la  |  /la/
  <<<<<<<<>>>>>>>>>
   vida  |  /biða/

  phrase>> ya teng